In [1]:
import folium
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib3
import webbrowser
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
metro_df = pd.read_csv('metro.tsv', sep='\t')
print(metro_df.head())

              Название    Широта   Долгота
0             Аэропорт  55.79981  37.53412
1        Академическая  55.68808  37.57501
2         Алексеевская  55.80737  37.63844
3  Александровский сад  55.75219  37.60836
4            Алтуфьево  55.89504  37.58605


In [2]:
def find_metros(url: str) -> list:
    try:
        response = requests.get(url, verify=False)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f'Не удалось получить данные по URL {url}: {e}')
        return []
    soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'), 'html.parser')
    stations = []
    for div in soup.find_all('div', class_='features-item'):
        title = div.find('span', class_='features-title')
        if title:
            text = title.text.rstrip(',').strip()
            stations.append(text)
    print(f'Найдено {len(stations)} станций на странице {url}.')
    return stations

In [3]:
def get_districts() -> dict:
    base_url = 'https://domokucha.info/moscow_metro_district.html'
    try:
        response = requests.get(base_url, verify=False)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f'Не удалось получить главную страницу районов: {e}')
        return {}
    soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'), 'html.parser')
    districts = {} 
    for item in soup.find_all('div', class_='features-item'):
        link = item.find('a', href=True)
        if link and link['href'].startswith('./moscow_metro'):
            district_name = item.find('span', class_='features-title').text.strip()
            metro_page_url = urljoin(base_url, link['href'])
            stations = find_metros(metro_page_url)
            for station in stations:
                districts[station.lower()] = district_name
    print(f'Сопоставлено {len(districts)} станций с районами.')
    return districts

In [4]:
def generate_map(data: pd.DataFrame, districts: dict, map_file: str) -> folium.Map:
    moscow_coords = [55.751244, 37.618423]
    metro_map = folium.Map(location=moscow_coords, zoom_start=10)
    district_colors = {
        'Восточный административный округ': 'green',
        'Западный административный округ': 'orange',
        'Северный административный округ': 'red',
        'Северо-Западный административный округ': 'blue',
        'Северо-Восточный административный округ': 'pink',
        'Центральный административный округ': 'purple',
        'Юго-Восточный административный округ': 'beige',
        'Юго-Западный административный округ': 'darkblue',
        'Южный административный округ': 'yellow'
    }
    for _, row in data.iterrows():
        station = row.get('Название', '').strip()
        if not station:
            continue
        district = districts.get(station.lower(), 'Не найдено')
        color = district_colors.get(district, 'lightgray')
        lat = row.get('Широта')
        lon = row.get('Долгота')
        if pd.isnull(lat) or pd.isnull(lon):
            print(f'Пропуск станции {station} из-за отсутствия координат.')
            continue
        folium.Marker(
            location=[lat, lon],
            popup=f'{station} ({lat}, {lon}) - {district}',
            icon=folium.Icon(color=color)
        ).add_to(metro_map)
    metro_map.save(map_file)
    print(f'Карта сохранена в {map_file}.')
    return metro_map

In [5]:
def main():
    map_file = 'map.html'
    districts = get_districts()
    if not districts:
        print('Отсутствуют данные о районах.')
        return
    metro_map = generate_map(metro_df, districts, map_file)
    try:
        webbrowser.open(map_file)
        print('Открытие карты в веб-браузере.')
    except Exception as e:
        print(f'Не удалось открыть карту в браузере: {e}')
if __name__ == "__main__":
    main()

Найдено 14 станций на странице https://domokucha.info/moscow_metro_01_VAO.html.
Найдено 22 станций на странице https://domokucha.info/moscow_metro_02_ZAO.html.
Найдено 26 станций на странице https://domokucha.info/moscow_metro_03_SAO.html.
Найдено 12 станций на странице https://domokucha.info/moscow_metro_04_SZAO.html.
Найдено 26 станций на странице https://domokucha.info/moscow_metro_05_SVAO.html.
Найдено 65 станций на странице https://domokucha.info/moscow_metro_06_CAO.html.
Найдено 17 станций на странице https://domokucha.info/moscow_metro_07_UVAO.html.
Найдено 19 станций на странице https://domokucha.info/moscow_metro_08_UZAO.html.
Найдено 25 станций на странице https://domokucha.info/moscow_metro_09_UAO.html.
Найдено 2 станций на странице https://domokucha.info/moscow_metro_10_Zelenograd.html.
Найдено 9 станций на странице https://domokucha.info/moscow_metro_11_New-moscow.html.
Найдено 2 станций на странице https://domokucha.info/moscow_metro_12_Troicky.html.
Найдено 2 станций на 

C:\Users\utkad\AppData\Local\Temp\ipykernel_12072\2195191535.py:29: UserWarning: color argument of Icon should be one of: {'darkred', 'lightgray', 'orange', 'green', 'lightblue', 'lightred', 'pink', 'blue', 'black', 'lightgreen', 'darkpurple', 'purple', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'red', 'white', 'gray'}.
  icon=folium.Icon(color=color)


Карта сохранена в map.html.
Открытие карты в веб-браузере.
